In [2]:
from util import *

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# this block of codes comes from Nilavra Bhattacharya
# http://nbviewer.jupyter.org/urls/ivc.ischool.utexas.edu/VizWiz/extn/tutorial_skill_type.ipynb

df_train = pd.read_csv('../vizwiz_skill_typ_train.csv', skipinitialspace = True, engine='python')
df_val = pd.read_csv('../vizwiz_skill_typ_val.csv', skipinitialspace = True, engine='python')
df_test = pd.read_csv('../vizwiz_skill_typ_test.csv', skipinitialspace = True, engine='python')

In [4]:
# check class distributions
target_classes = ['TXT', 'OBJ', 'COL', 'CNT', 'OTH']
class_dist = []
for skill in target_classes:
    percent = len(df_train.loc[df_train[skill] == 1])/len(df_train)
    temp = {'skill': skill, '% of true': percent}
    class_dist.append(temp)
class_dist = pd.DataFrame(class_dist)
class_dist

,% of true,skill
0,0.561847,TXT
1,0.984652,OBJ
2,0.379564,COL
3,0.036513,CNT
4,0.001542,OTH


In [5]:
# correlations: OBJ is correlated with COL
df_train[['TXT', 'OBJ', 'COL', 'CNT', 'OTH']].corr()

,TXT,OBJ,COL,CNT,OTH
TXT,1.000000,0.031074,-0.479960,0.094344,-0.019298
OBJ,0.031074,1.000000,-0.035096,0.006068,0.004906
COL,-0.479960,-0.035096,1.000000,-0.106065,-0.016013
CNT,0.094344,0.006068,-0.106065,1.000000,-0.007650
OTH,-0.019298,0.004906,-0.016013,-0.007650,1.000000


In [12]:
# get object labels with VGG and AlexNet pre-trained on ImageNet
nn_features_test = nn_feature_extract(df_test, cv_key, 'nn_features_test.dat', test_data=True)
nn_features_test.to_csv("nn_features_test.csv")
print(nn_features_test.shape, df_test.shape)

(5721, 3) (5721, 3)


In [17]:
nn_features_val = nn_feature_extract(df_val, cv_key, 'nn_features_val.dat', test_data=False)
nn_features_test.to_csv("nn_features_val.csv")
print(nn_features_val.shape, df_val.shape)

(2251, 5) (2251, 19)


In [19]:
nn_features_train = nn_feature_extract(df_train, cv_key, 'nn_features_train.dat', test_data=False)
nn_features_train.to_csv("nn_features_train.csv")
print(nn_features_train.shape, df_train.shape)

(14269, 5) (14269, 19)


In [12]:
# read API features from csv file 
train_feature_api = pd.read_csv("api_features_train.csv")
train_feature_nn = pd.read_csv("nn_features_train.csv")

In [16]:
train_feature_nn.head(2)

,qid,vgg_tags alexnet_tags,txt,col
0,VizWiz_train_000000000000.jpg,perfume essence packet,1,0
1,VizWiz_train_000000000001.jpg,espresso maker lighter light igniter ignitor,1,0


In [20]:
train_feature_api.sample(2)

,qid,adult,categories,descriptions,tags,adult.1,categories.1,descriptions.1,tags.1,col,txt,adult.2,categories.2,descriptions.2,tags.2
5377,VizWiz_train_000000007182.jpg,False,others_,None,'indoor' 'table' 'sitting' 'monitor' 'devi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11864,VizWiz_train_000000015879.jpg,False,abstract_,a close up of a white wall,'table' 'laying' 'cat' 'white' 'bed' 'man',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def aggregate_features(nn_features, api_features, test_data=False):
    nn_features.set_index('qid', inplace=True)
    api_features.set_index('qid', inplace=True)
    merged = nn_features.join(api_features)
    if not test_data:
        # merged.drop([['txt', 'col']])   # drop duplicate cols
    print("merged feature set size:", merged.shape)
    print(merged.head(3))
    return merged